In [1]:
from transformers import AutoModel, AutoTokenizer, AutoModelForTokenClassification

In [36]:
checkpoint_path = '/home/ksaputa/mspace/plotkarzyna/models/herbert-large/checkpoint-2080'
model = AutoModelForTokenClassification.from_pretrained(checkpoint_path)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path, return_tensors='pt')

In [37]:
id2label = {
    0: "O",
    1: "B",
    2: "I",
    3: "E",
    4: "S"
}

In [38]:
tokenizer("Ala ma kota, który wszedł na drzewo.")

{'input_ids': [0, 37, 2121, 2185, 24112, 1947, 2377, 12745, 1998, 16621, 1899, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [78]:
def rindex(lst, value='B'):
    lst.reverse()
    try:
        i = lst.index(value)
    except ValueError:
        return 0
    else:
        lst.reverse()
        index = len(lst) - i - 1
        if index in range(len(lst)):
            return index
        else:
            return 0


def decode_bioes(labels, tokens):
    spans = []
    curr_spans = []
    curr_spans_types = []
    for ind, (token, token_label) in enumerate(zip(tokens, labels)):
        if token_label == 'O':
            if curr_spans:
                spans.extend(curr_spans)
                curr_spans = []
                curr_spans_types = []
            else:
                continue
        else:
            for curr_span in curr_spans:
                curr_span.append(token)

            if token_label in ['B', 'S']:
                if curr_spans_types and curr_spans_types[-1] == 'S':
                    continue
                else:
                    curr_spans_types.append(token_label)
                    curr_spans.append([token])
            elif token_label == 'I':
                continue
            elif token_label == 'E':
                next_label = labels[ind + 1] if len(labels) - 1 > ind else None
                if next_label and next_label == 'E':
                    # print(curr_spans, curr_spans_types)
                    curr_spans_types.append(token_label)
                    continue

                span_to_finish_ind = rindex(curr_spans_types)
                # print(curr_spans, curr_spans_types, span_to_finish_ind)
                if curr_spans_types and curr_spans:
                    try:
                        curr_spans_types.pop(span_to_finish_ind)
                        span_to_finish = curr_spans.pop(span_to_finish_ind)
                        spans.append(span_to_finish)
                    except IndexError:
                        pass
    spans.extend(curr_spans)
    return spans


In [79]:
text = "Ala ma kota, który wszedł na drzewo."
text = "Pogoda pod koniec tygodnia nie będzie miała dla nas litości. W dalszym ciągu spodziewamy się upałów. Miejscami temperatura zdecydowanie przekroczy 30 st. C i tylko lokalnie orzeźwienie przyniosą burze."
text = "Koalicja Obywatelska już odkryła karty i wiemy, jakie nazwiska znajdą się na listach w poszczególnych okręgach podczas jesiennych wyborów do Sejmu. Teraz czas na Lewicę. Partia ujawnia swoich kandydatów."
text = """Jeżeli pole to jest polem centralnym, zależnym tylko od odległości cząstek, a niezależnym od kierunku w przestrzeni, to teoria kwantowa przewiduje, że stan o najniższej energii winien mieć zerowy moment pędu i symetrię sferyczną. Tymczasem pomiary rozkładu ładunku elektrycznego w deuteronie wskazują na wyraźne, jakkolwiek niewielkie odstępstwa od takiej kulistej symetrii. Co więcej, nie można było pogodzić własności magnetycznych deuteronu z własnościami magnetycznymi jego składników. Ponieważ spin deuteronu wynosi 1, więc połówkowe spiny neutronu i protonu muszą być w nim ustawione równolegle. Kierunki wektorów momentów magnetycznych związane są z kierunkami spinów, zatem moment magnetyczny deuteronu winien być sumą momentów protonu i neutronu. Zaobserwowane odstępstwa świadczą o tym, że pewien przyczynek do momentu magnetycznego deuteronu musi pochodzić z ruchu protonu po orbicie z momentem pędu większym od zera. Ponieważ w przypadku sił centralnych stan o najniższej energii musi mieć moment pędu równy zeru, możemy wyciągnąć stąd wniosek o niecentralności sił jądrowych. Muszą one, poza zależnością od odległości nukleonów, zależeć w jakiś sposób od kierunku w przestrzeni. Skąd jednak taka zależność może pochodzić?

Oddziaływania między nukleonami mogą zależeć również od innych poza ich odległością cech nukleonów, takich jak pęd czy moment pędu w ich ruchu, a także ich spiny i izospiny. Jest oczywiste, że mając do dyspozycji obok wektora odległości R również wektory spinów nukleonów S1 i S2, możemy część niecentralną oddziaływania utworzyć jako zależną od wzajemnej orientacji tych wektorów (rys.4.4). Skąd jednak znaleźć możemy postać tej zależności?

"""

tokenized = tokenizer([text], return_tensors='pt')
pred = model(
    **tokenized
).logits.argmax(-1)[0]

print(' '.join([
    f"|{tokenizer.decode(tok, clean_up_tokenization_spaces=False)}| ({id2label[int(el)]})" for el, tok in zip(pred, list(tokenized['input_ids'][0]))
]))
tokens = [tokenizer.decode(tok) for tok in list(tokenized['input_ids'][0])]
labels = [id2label[int(el)] for el, tok in zip(pred, list(tokenized['input_ids'][0]))]
decode_bioes(labels, tokens)

|<s>| (O) |Jeżeli| (O) |pole| (B) |to| (E) |jest| (O) |polem| (B) |centralnym| (I) |,| (I) |zależ| (I) |nym| (I) |tylko| (I) |od| (I) |odległości| (B) |cząstek| (S) |,| (I) |a| (I) |niezależ| (I) |nym| (I) |od| (I) |kierunku| (B) |w| (I) |przestrzeni| (S) |,| (O) |to| (O) |teoria| (B) |kwan| (E) |towa| (E) |przewiduje| (O) |,| (O) |że| (O) |stan| (B) |o| (I) |najniż| (B) |szej| (O) |energii| (E) |winien| (O) |mieć| (O) |zer| (B) |owy| (B) |moment| (I) |pędu| (S) |i| (I) |sy| (B) |met| (I) |rię| (B) |s| (E) |fery| (E) |czną| (O) |.| (O) |Tymczasem| (O) |pomiary| (B) |rozkładu| (B) |ładunku| (B) |elektrycznego| (E) |w| (I) |de| (S) |u| (S) |tero| (E) |nie| (E) |wskazują| (O) |na| (O) |wyraźne| (B) |,| (I) |jakkolwiek| (I) |niewielkie| (I) |odstęp| (I) |stwa| (I) |od| (I) |takiej| (B) |ku| (I) |list| (I) |ej| (I) |sy| (E) |metrii| (E) |.| (O) |Co| (O) |więcej| (O) |,| (O) |nie| (O) |można| (O) |było| (O) |pogodzić| (O) |własności| (B) |magne| (I) |tycznych| (I) |de| (S) |u| (S) |te| (S) |

[['pole', 'to'],
 ['polem',
  'centralnym',
  ',',
  'zależ',
  'nym',
  'tylko',
  'od',
  'odległości',
  'cząstek',
  ',',
  'a',
  'niezależ',
  'nym',
  'od',
  'kierunku',
  'w',
  'przestrzeni'],
 ['odległości',
  'cząstek',
  ',',
  'a',
  'niezależ',
  'nym',
  'od',
  'kierunku',
  'w',
  'przestrzeni'],
 ['cząstek',
  ',',
  'a',
  'niezależ',
  'nym',
  'od',
  'kierunku',
  'w',
  'przestrzeni'],
 ['teoria', 'kwan', 'towa'],
 ['stan', 'o', 'najniż'],
 ['najniż'],
 ['owy', 'moment', 'pędu', 'i', 'sy', 'met', 'rię', 's', 'fery'],
 ['zer', 'owy', 'moment', 'pędu', 'i', 'sy', 'met', 'rię', 's', 'fery'],
 ['pędu', 'i', 'sy', 'met', 'rię', 's', 'fery'],
 ['ładunku', 'elektrycznego'],
 ['rozkładu', 'ładunku', 'elektrycznego', 'w', 'de', 'u', 'tero', 'nie'],
 ['pomiary',
  'rozkładu',
  'ładunku',
  'elektrycznego',
  'w',
  'de',
  'u',
  'tero',
  'nie'],
 ['de', 'u', 'tero', 'nie'],
 ['takiej', 'ku', 'list', 'ej', 'sy', 'metrii'],
 ['wyraźne',
  ',',
  'jakkolwiek',
  'niewielk

In [ ]:
decode_bioes(labels, tokens)

[['Koalicja', 'Obywatelska']] ['B'] 0
[['listach', 'w', 'poszczególnych', 'okręgach'], ['poszczególnych', 'okręgach']] ['S', 'B'] 1
[['swoich', 'kandydatów']] ['B'] 0


[['Koalicja', 'Obywatelska'],
 ['karty'],
 ['wiemy'],
 ['nazwiska'],
 ['poszczególnych', 'okręgach'],
 ['listach', 'w', 'poszczególnych', 'okręgach'],
 ['jesie', 'nnych', 'wyborów', 'do', 'Sejmu'],
 ['nnych', 'wyborów', 'do', 'Sejmu'],
 ['Sejmu'],
 ['Lewi', 'cę'],
 ['cę'],
 ['Partia'],
 ['swoich', 'kandydatów']]

In [6]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), 